In [1]:
import os
import torch
from tqdm import tqdm
import plotly.express as px

torch.set_grad_enabled(False)

In [2]:
# For the most part I'll try to import functions and classes near where they are used
# to make it clear where they come from.

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

Device: cuda


In [3]:
from datasets import load_dataset
from transformer_lens import HookedTransformer
from sae_lens import SAE

model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience.
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="gpt2-small-res-jb",  # see other options in sae_lens/pretrained_saes.yaml
    sae_id="blocks.8.hook_resid_pre",  # won't always be a hook point
    device=device,
)

Loaded pretrained model gpt2-small into HookedTransformer


/opt/conda/lib/python3.10/site-packages/sae_lens/sae.py:136: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


In [21]:
# def zero_abl_hook(activation, hook):
#     return torch.zeros_like(activation)

def direct(activation, hook):
    return activation

def clamping(activation, hook):
    encoded = sae.encode(activation)

    # The activations are now stored in intermediate_outputs
    # Modify activations
    # print(encoded.shape)
    # print(encoded[:, :, torch.argmax(encoded)])
    # encoded[:, :, :] = 0.0
    encoded[0, :, 123] = 50.0

    sae_out = sae.decode(encoded)

    return sae_out

In [22]:
example_prompt = "It is safest when the flow in a pipe is"

with model.hooks(
    fwd_hooks=[(sae.cfg.hook_name, direct)],
):
    
    input_tokens = model.to_tokens(example_prompt, prepend_bos=True)

    generated_tokens = model.generate(input_tokens, max_new_tokens=100, do_sample=True)

    generated_tokens = model.tokenizer.decode(generated_tokens[0])
    print(generated_tokens)

with model.hooks(
    fwd_hooks=[(sae.cfg.hook_name, clamping)],
):
    input_tokens = model.to_tokens(example_prompt, prepend_bos=True)

    generated_tokens = model.generate(input_tokens, max_new_tokens=100, do_sample=True)

    generated_tokens = model.tokenizer.decode(generated_tokens[0])
    print(generated_tokens)



  0%|          | 0/100 [00:00<?, ?it/s]

<|endoftext|>It is safest when the flow in a pipe is relatively high and continuous, such as when welding up steel, providing flexible welds and the than midway impact, where resistance is very high. FUSE operates primarily because of its high hydraulic strength, which means that over suction induced pressure is not communicated to resistive material. This makes FUSE safer to use when welding blocks on other types of pipes, especially those that have originated on pipes that draw less than one-half inch downward. Smaller blocks, or presses, or pipes with lower


  0%|          | 0/100 [00:00<?, ?it/s]

<|endoftext|>It is safest when the flow in a pipe is crowded and affordable to Stone's Weekly Harvest

Camp Phays Central Middle School hosted The Student Center North Center

Anti-Hot Spring Music Festival begins

Phary Ball Sony Music Festival, TBA

Crystal Creek In 844 Distillier Good Central Per Fill Cango De Praeda Print Garden The Big East ALFC The Cellos Amphitheater Amphitheatre Diana Arena Caes Clayer from The Bottom Down Auditorium

American Renaissance Festival Chesapeake Children's Coliseum


